In [4]:
import pandas as pd
!pip install psycopg2
import psycopg2
# !pip install sqlalchemy
import sqlalchemy
from sqlalchemy import create_engine
import numpy as np
# !pip install seaborn
import seaborn as sns
# !pip install pathlib
import pathlib



[notice] A new release of pip is available: 22.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# !pip install SQLAlchemy


[notice] A new release of pip is available: 22.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import sqlalchemy
sqlalchemy.__version__  

'2.0.9'

In [9]:
cred = {
    'host': 'ep-red-bar-a1rghkov.ap-southeast-1.aws.neon.tech',
    'user': 'siswa_bfp',
    'pass': 'bfp_aksel_keren',
    'db': 'ecommerce_db',
    'port': 5432
}

uri = f"postgresql+psycopg2://{cred['user']}:{cred['pass']}\
@{cred['host']}:{cred['port']}/{cred['db']}"

conn = create_engine(uri)
conn.connect()


In [7]:
from sqlalchemy import inspect
inspector = inspect(conn)

for table_name in inspector.get_table_names():
    print("Table: %s" % table_name)
    for column in inspector.get_columns(table_name):
        print("Column: %s" % column['name'])
    print('\n')

Table: customers
Column: cust_id
Column: name
Column: phone
Column: address
Column: city
Column: province


Table: carts
Column: cart_id
Column: cust_id
Column: trx_date


Table: cartitems
Column: cart_item_id
Column: cart_id
Column: product_id
Column: order_amt


Table: products
Column: product_id
Column: name
Column: price




In [12]:
df_customers = pd.read_sql_table(table_name = 'customers',con = conn.connect())
df_carts = pd.read_sql_table(table_name = 'carts',con = conn.connect())
df_cartitems = pd.read_sql_table(table_name = 'cartitems',con = conn.connect())
df_products = pd.read_sql_table(table_name = 'products',con = conn.connect())


In [14]:
# tutup koneksi ke database
conn.dispose()

In [26]:
xlsx_url = 'https://raw.githubusercontent.com/Kurikulum-Sekolah-Pacmann/wrangling-bfp-aksel/refs/heads/main/ecommerce_data/Campaigns.xlsx' 
json_url = 'https://raw.githubusercontent.com/Kurikulum-Sekolah-Pacmann/wrangling-bfp-aksel/refs/heads/main/ecommerce_data/Products.json' 
df_campaign = pd.read_excel(xlsx_url)
df_catalogue = pd.read_json(json_url)

In [197]:
df_catalogue_products = pd.merge(df_products,df_catalogue,on=['product_id','name','price'])
table_products = df_catalogue_products[['product_id', 'name', 'price', 'description', 'category', 'image_url']]
table_products['category'] = table_products['category'].map(lambda x:x['category_id'])
table_products.rename(columns={'category':'category_id'},inplace=True)
table_products

C:\Users\msing\AppData\Local\Temp\ipykernel_23080\3582221351.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_products['category'] = table_products['category'].map(lambda x:x['category_id'])
C:\Users\msing\AppData\Local\Temp\ipykernel_23080\3582221351.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_products.rename(columns={'category':'category_id'},inplace=True)


,product_id,name,price,description,category_id,image_url
0,1,Japchae,72500,A Korean dish made with stir-fried sweet potat...,1,https://fake.pacmann.io/img/1.jpg
1,2,Frozen Tater Tots,35000,"Small, grated potatoes formed into rounds and ...",2,https://fake.pacmann.io/img/2.jpg
2,3,Frozen Hot Pockets,32000,Microwaveable pastries filled with a variety o...,2,https://fake.pacmann.io/img/3.jpg
3,4,Duck Foie Gras,112500,A luxurious French delicacy made from the live...,3,https://fake.pacmann.io/img/4.jpg
4,5,Mee Goreng,66000,A flavorful and spicy stir-fried noodle dish f...,1,https://fake.pacmann.io/img/5.jpg
...,...,...,...,...,...,...
147,148,Chicken Quesadilla,108000,A Mexican dish made by stuffing a tortilla wit...,3,https://fake.pacmann.io/img/148.jpg
148,149,Egg Noodles with Stir-Fry Vegetables,64500,A simple Chinese dish made with soft egg noodl...,1,https://fake.pacmann.io/img/149.jpg
149,150,Frozen Breakfast Sandwich,53000,Pre-made breakfast sandwiches filled with eggs...,2,https://fake.pacmann.io/img/150.jpg
150,151,Blueberry Smoothie,21500,"A blend of blueberries, yogurt, and a touch of...",5,https://fake.pacmann.io/img/151.jpg


In [143]:
table_categories = pd.DataFrame(df_catalogue_products['category'].to_list()).drop_duplicates().reset_index(drop=True)[['category_id','name']]
table_categories

,category_id,name
0,1,noodles
1,2,frozen
2,3,chicken_duck
3,4,coffee
4,5,beverages
5,6,rice


In [209]:
df_tags = df_catalogue_products[['product_id','tags']]

tag_index = {value:str(key+1) for key,value in enumerate(sorted(df_tags.explode('tags')['tags'].unique()))}
df_tags = df_tags.explode('tags').reset_index(drop=True)
# df_tags['tag_id'] = df_tags['tags'].map(tag_index)
df_tags['tag_id'] = df_tags['product_id'].astype('str')
df_tags['tag_id'] = df_tags['tag_id'] +"-"+ df_tags['tags'].map(tag_index)
df_tags = df_tags.rename(columns={'tags':'tag'})
table_tags = df_tags[['tag_id','tag','product_id']]
table_tags



,tag_id,tag,product_id
0,1-280,potato,1
1,1-328,seasonings,1
2,1-369,stir-fried,1
3,1-185,korean,1
4,1-24,beef,1
...,...,...,...
656,152-72,comfort,152
657,152-136,food,152
658,152-57,cheese,152
659,152-82,creamy,152


In [142]:
table_campaigns = df_campaign[['campaign_id','name','campaign_type','start_date']]
table_campaigns


,campaign_id,name,campaign_type,start_date
0,1,Festival Makanan Sehat,Category,2023-01-01
1,2,Bulan Cita Rasa Nusantara,Product,2023-02-01
2,3,Mei Festival Makanan Lokal,Category,2023-05-01
3,4,Festival Pedas Juli,Category,2023-07-01
4,5,Bulan Makanan Manis,Category,2023-09-01
5,6,November Nikmat,Product,2023-11-01
6,7,Festival Makanan Laut Februari,Product,2024-02-01
7,8,Maret Rasa Pedas,Product,2024-03-01
8,9,Juni Hidangan Segar,Category,2024-06-01
9,10,Agustus Kuliner Kreatif,Product,2024-08-01


In [141]:
table_campaign_items = df_campaign[['campaign_id','campaign_type','campaign_objects']].copy()
table_campaign_items['campaign_objects'] = table_campaign_items['campaign_objects'].map(lambda x:x.split(';'))
table_campaign_items = table_campaign_items.explode('campaign_objects').reset_index(drop=True)
cat_mapper = dict(zip(table_categories['category_id'],table_categories['name']))
prd_mapper = dict(zip(df_products['product_id'],df_products['name']))
table_campaign_items['campaign_item'] = [cat_mapper[int(i[2])] if i[1]=='Category' else prd_mapper[int(i[2])] for i in table_campaign_items[['campaign_type','campaign_objects']].itertuples()]
table_campaign_items = table_campaign_items.reset_index(names='campaign_item_id').drop(columns=['campaign_objects'])
table_campaign_items

,campaign_item_id,campaign_id,campaign_type,campaign_item
0,0,1,Category,frozen
1,1,1,Category,chicken_duck
2,2,1,Category,rice
3,3,2,Product,Chicken Shawarma
4,4,2,Product,Frozen Hash Browns
5,5,2,Product,Iced Latte
6,6,2,Product,Duck Breast with Cherry Sauce
7,7,2,Product,Macchiato
8,8,2,Product,Frozen Lasagna
9,9,3,Category,noodles


### SQLAlchemy classes part

In [146]:
import hashlib
import typing
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.orm import (
    declarative_base,
    mapped_column,
    relationship,
    sessionmaker,
    Mapped,
)

In [210]:
from __future__ import annotations
import hashlib
import typing
import sqlalchemy as sa
from typing import List
from typing import Optional
from sqlalchemy import create_engine
from sqlalchemy import ForeignKey
from sqlalchemy import Integer
from sqlalchemy.orm import (
    DeclarativeBase,
    mapped_column,
    relationship,
    sessionmaker,
    Mapped,
)

cred = {
    'host': 'ep-square-pond-a5h66kcl.us-east-2.aws.neon.tech',
    'user': 'neondb_owner',
    'pass': 'Kd2w6OvIiUtr',
    'db': 'neondb',
    'port': 5432
}



uri = f"postgresql+psycopg2://{cred['user']}:{cred['pass']}\
@{cred['host']}:{cred['port']}/{cred['db']}"

conn = create_engine(uri,echo=True)
# conn.connect()


class Base(DeclarativeBase):
    pass


class Campaigns(Base):
    __tablename__ = "campaigns"

    campaign_id : Mapped[int] = mapped_column(primary_key=True)
    name        : Mapped[str] = mapped_column(sa.String)
    campaign_type: Mapped[str] = mapped_column(sa.String)
    start_date  : Mapped[str] = mapped_column(sa.String(30))
    
    campitems_camp: Mapped[List['CampaignItems']] = relationship(back_populates='camp_campitems')

    # def __repr__(self) -> str:
    #     return f"<User(username={self.auth.username}, email={self.auth.email})>"

class CampaignItems(Base):
    __tablename__ = "campaign_items"

    campaign_item_id: Mapped[int] = mapped_column(primary_key=True)
    campaign_id     : Mapped[int] = mapped_column(ForeignKey("campaigns.campaign_id"))
    campaign_type   : Mapped[str] = mapped_column(sa.String)
    campaign_item   : Mapped[str] = mapped_column(sa.String)

    camp_campitems: Mapped['Campaigns'] = relationship(back_populates='campitems_camp')

class Products(Base):
    __tablename__ = "products"

    product_id  : Mapped[int] = mapped_column(primary_key=True)
    name        : Mapped[str] = mapped_column(sa.String)
    price       : Mapped[int] = mapped_column(sa.Float)
    description : Mapped[str] = mapped_column(sa.String)
    category_id : Mapped[int] = mapped_column(ForeignKey('categories.category_id'))
    image_url   : Mapped[str] = mapped_column(sa.String)

    categories_product  : Mapped['Categories'] = relationship(back_populates='product_category')
    tags_product        : Mapped[List['Tags']] = relationship(back_populates='product_tags')
    cartitems_products  : Mapped[List['CartItems']] = relationship(back_populates='products_cartitems')

    # def __repr__(self) -> str:
    #     return f"<User(username={self.auth.username}, email={self.auth.email})>"

class Categories(Base):
    __tablename__ = "categories"

    category_id : Mapped[int] = mapped_column(primary_key=True)
    name        : Mapped[str] = mapped_column(sa.String)
    
    product_category: Mapped[List['Products']] = relationship(back_populates='categories_product')

class Tags(Base):
    __tablename__ = "tags"

    tag_id      : Mapped[str] = mapped_column(primary_key=True)
    tag         : Mapped[str] = mapped_column(sa.String)
    product_id  : Mapped[int] = mapped_column(ForeignKey('products.product_id'))
    
    
    product_tags: Mapped['Products'] = relationship(back_populates='tags_product')


class Customers(Base):
    __tablename__ = "customers"

    cust_id     : Mapped[int] = mapped_column(primary_key=True)
    name        : Mapped[str] = mapped_column(sa.String)
    phone       : Mapped[str] = mapped_column(sa.String(30))
    address     : Mapped[str] = mapped_column(sa.String)
    city        : Mapped[str] = mapped_column(sa.String(30))
    province    : Mapped[str] = mapped_column(sa.String(30))
    
    cart_customer  : Mapped[List['Carts']] = relationship(back_populates='customer_cart')
    

class Carts(Base):
    __tablename__ = "carts"

    cart_id     : Mapped[int] = mapped_column(primary_key=True)
    cust_id     : Mapped[int] = mapped_column(ForeignKey('customers.cust_id'))
    trx_date    : Mapped[str] = mapped_column(sa.String)
    
    customer_cart  : Mapped['Customers'] = relationship(back_populates='cart_customer')
    cartitems_carts  : Mapped[List['CartItems']] = relationship(back_populates='carts_cartitems')


class CartItems(Base):
    __tablename__ = "cart_items"

    cart_item_id: Mapped[int] = mapped_column(primary_key=True)
    cart_id     : Mapped[int] = mapped_column(ForeignKey('carts.cart_id'))
    product_id  : Mapped[int] = mapped_column(ForeignKey('products.product_id'))
    order_amt   : Mapped[int] = mapped_column(sa.Integer)
    
    carts_cartitems  : Mapped['Carts'] = relationship(back_populates='cartitems_carts')
    products_cartitems: Mapped['Products'] = relationship(back_populates='cartitems_products')



Base.metadata.create_all(conn)

2024-12-11 14:49:50,389 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-12-11 14:49:50,390 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-11 14:49:51,092 INFO sqlalchemy.engine.Engine select current_schema()
2024-12-11 14:49:51,094 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-11 14:49:51,894 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-12-11 14:49:51,897 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-11 14:49:52,922 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-11 14:49:52,929 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [ ]:



cred = {
    'host': 'ep-square-pond-a5h66kcl.us-east-2.aws.neon.tech',
    'user': 'neondb_owner',
    'pass': 'Kd2w6OvIiUtr',
    'db': 'neondb',
    'port': 5432
}



uri = f"postgresql+psycopg2://{cred['user']}:{cred['pass']}\
@{cred['host']}:{cred['port']}/{cred['db']}"

conn = create_engine(uri,echo=True)


table_campaigns.to_sql(name = 'campaigns',con = conn,if_exists = "append",index = False)
table_campaign_items.to_sql(name = 'campaign_items',con = conn,if_exists = "append",index = False)
table_categories.to_sql(name = 'categories',con = conn,if_exists = "append",index = False)
df_customers.to_sql(name = 'customers',con = conn,if_exists = "append",index = False)
df_carts.to_sql(name = 'carts',con = conn,if_exists = "append",index = False)
table_products.to_sql(name = 'products',con = conn,if_exists = "append",index = False)
table_tags.to_sql(name = 'tags',con = conn,if_exists = "append",index = False)
df_cartitems.to_sql(name = 'cart_items',con = conn,if_exists = "append",index = False)


2024-12-11 14:50:01,408 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-12-11 14:50:01,411 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-11 14:50:01,951 INFO sqlalchemy.engine.Engine select current_schema()
2024-12-11 14:50:01,953 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-11 14:50:02,638 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-12-11 14:50:02,639 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-11 14:50:03,360 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-11 14:50:03,361 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

871

# ANALYTICS

In [3]:
import pandas as pd
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine
import numpy as np
import seaborn as sns
import pathlib


cred = {
    'host': 'ep-square-pond-a5h66kcl.us-east-2.aws.neon.tech',
    'user': 'neondb_owner',
    'pass': 'Kd2w6OvIiUtr',
    'db': 'neondb',
    'port': 5432
}



uri = f"postgresql://{cred['user']}:{cred['pass']}\
@{cred['host']}:{cred['port']}/{cred['db']}"

conn = create_engine(uri,echo=True)

df_customers = pd.read_sql_table(table_name = 'customers',con = conn.connect())
df_carts = pd.read_sql_table(table_name = 'carts',con = conn.connect())
df_cartitems = pd.read_sql_table(table_name = 'cart_items',con = conn.connect())
df_products = pd.read_sql_table(table_name = 'products',con = conn.connect())
df_campaign = pd.read_sql_table(table_name = 'campaigns',con = conn.connect())
df_campaign_items = pd.read_sql_table(table_name = 'campaign_items',con = conn.connect())
df_categories = pd.read_sql_table(table_name = 'categories' ,con = conn.connect())
df_tags = pd.read_sql_table(table_name = 'tags',con = conn.connect())


2024-12-12 09:37:51,771 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-12-12 09:37:51,773 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-12 09:37:52,310 INFO sqlalchemy.engine.Engine select current_schema()
2024-12-12 09:37:52,311 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-12 09:37:52,769 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-12-12 09:37:52,772 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-12 09:37:53,233 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-12 09:37:53,235 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [29]:
# Top product berdasar unit terjual

top_product = df_cartitems.groupby('product_id')
top_product = df_products[['product_id','name']].set_index('product_id').join(pd.DataFrame(top_product.sum()['order_amt'].sort_values(ascending=False)),how='right').reset_index().iloc[0,:]
top_product


product_id         24
name          Biryani
order_amt        2443
Name: 0, dtype: object

,cart_id,cust_id,trx_date,cart_item_id,product_id,order_amt
0,5,812,2023-01-01 00:00:00,14,24,1
1,9,549,2023-01-01 00:00:00,29,24,3
2,22,76,2023-01-03 00:00:00,73,24,2
3,41,210,2023-01-05 00:00:00,131,24,1
4,42,548,2023-01-05 00:00:00,136,24,3
...,...,...,...,...,...,...
1207,9247,679,2024-11-27 00:00:00,28746,24,2
1208,9259,334,2024-11-28 00:00:00,28786,24,1
1209,9262,73,2024-11-28 00:00:00,28794,24,3
1210,9268,844,2024-11-28 00:00:00,28812,24,2


In [53]:
# customer yang paling sering beli top produk
top_product_cust = pd.merge(df_carts,df_cartitems[df_cartitems['product_id']==top_product['product_id']],how='right',on='cart_id')
freq_cust_id = pd.DataFrame(top_product_cust['cust_id'].value_counts()).reset_index().rename(columns={'cust_id':'freq','index':'cust_id'}).iloc[0,:]['cust_id']
df_customers[df_customers['cust_id']==freq_cust_id]

,cust_id,name,phone,address,city,province
690,690,PT Nurdiyanti Anggraini Tbk,+62 (52) 429 5229,Jalan Suniaraja No. 6,Jakarta Timur,Jakarta


In [59]:
# kota asal customer yang paling sering melakukan pembelian atau transaksi
culprit = df_carts['cust_id'].value_counts().reset_index().rename(columns={'cust_id':'freq','index':'cust_id'}).iloc[0,:]['cust_id']
df_customers[df_customers['cust_id']==culprit]['city']

222    Jakarta Timur
Name: city, dtype: object

In [70]:
# customer yang melakukan repeat order (transaksi lebih dari 1)
id_cust = df_carts['cust_id'].value_counts().reset_index().rename(columns={'cust_id':'freq','index':'cust_id'})
list_id_cust = id_cust[id_cust['freq']>1]['cust_id'].tolist()
df_customers[df_customers['cust_id'].isin(list_id_cust)]

,cust_id,name,phone,address,city,province
0,1,PT Nuraini Tbk,(070) 602-8828,Gang Tebet Barat Dalam No. 5,Bekasi,Jawa Barat
1,2,PT Hariyah,(0130) 156 5916,Gg. Ciwastra No. 3,Bekasi,Jawa Barat
2,3,CV Farida Budiman Tbk,081 016 2783,Gg. Kiaracondong No. 217,Jakarta Timur,Jakarta
3,4,Perum Firmansyah (Persero) Tbk,+62-034-551-1097,Jalan Antapani Lama No. 256,Jakarta Pusat,Jakarta
4,5,PD Saefullah Lestari Tbk,(0208) 127 0881,Gang Rajawali Barat No. 152,Jakarta Barat,Jakarta
...,...,...,...,...,...,...
994,995,CV Mulyani,+62 (054) 203-3965,Jalan Surapati No. 4,Jakarta Selatan,Jakarta
995,996,Perum Andriani Yuniar,085 463 4021,Gg. Moch. Toha No. 2,Jakarta Utara,Jakarta
996,997,PD Puspasari (Persero) Tbk,+62 (39) 078-2523,Gang Indragiri No. 9,Jakarta Selatan,Jakarta
997,998,PT Nababan Hutapea,(063) 297 2763,Jl. Rajawali Timur No. 3,Depok,Jawa Barat


In [90]:
# jumlah transaksi paling tinggi yang harus dibayar customer  per transaksi (1 keranjang)
cart = pd.merge(df_carts,df_cartitems,how='left',on='cart_id')
cart_price = pd.merge(cart,df_products[['product_id','price']],on='product_id')
cart_price['total'] = cart_price['order_amt']*cart_price['price']
cart_price.groupby('cart_id')['total'].sum().sort_values(ascending=False).iloc[0]

1173000.0

In [97]:
df_products[df_products['product_id']==149]

,product_id,name,price,description,category_id,image_url
148,149,Egg Noodles with Stir-Fry Vegetables,64500.0,A simple Chinese dish made with soft egg noodl...,1,https://fake.pacmann.io/img/149.jpg


In [114]:
df_products.columns

Index(['product_id', 'name', 'price', 'description', 'category_id',
       'image_url'],
      dtype='object')

In [117]:
# produk yang paling mahal untuk tiap kategori produk
df_products.loc[df_products.groupby('category_id')['price'].idxmax()]

,product_id,name,price,description,category_id,image_url
16,17,Laksa,75500.0,"A spicy noodle soup popular in Southeast Asia,...",1,https://fake.pacmann.io/img/17.jpg
65,66,Frozen Vegetable Dumplings,70000.0,"Dumplings filled with vegetables or tofu, froz...",2,https://fake.pacmann.io/img/66.jpg
89,90,Chicken Biryani,118000.0,"A fragrant, spiced rice dish from South Asia m...",3,https://fake.pacmann.io/img/90.jpg
134,135,Iced Coffee,35500.0,Brewed coffee that is chilled and served over ...,4,https://fake.pacmann.io/img/135.jpg
22,23,Pu-erh Tea,39000.0,"A fermented tea from China, with a deep, earth...",5,https://fake.pacmann.io/img/23.jpg
54,55,Lemon Rice,70500.0,A South Indian dish where rice is flavored wit...,6,https://fake.pacmann.io/img/55.jpg


In [133]:
# produk favorit untuk tiap kategori produk berdasarkan jumlah transaksi
product_transaction = pd.merge(df_cartitems,df_products,on='product_id',how='left').groupby(['category_id','product_id'])['order_amt'].count().reset_index()
favorite_id = product_transaction.loc[product_transaction.groupby('category_id')['order_amt'].idxmax()]
pd.merge(favorite_id,df_products,on=['product_id','category_id'],how='left')


,category_id,product_id,order_amt,name,price,description,image_url
0,1,5,769,Mee Goreng,66000.0,A flavorful and spicy stir-fried noodle dish f...,https://fake.pacmann.io/img/5.jpg
1,2,119,640,Frozen Calzones,68000.0,Italian-style stuffed pizza pockets filled wit...,https://fake.pacmann.io/img/119.jpg
2,3,33,757,Duck Confit,105000.0,Duck legs slow-cooked in their own fat until t...,https://fake.pacmann.io/img/33.jpg
3,4,65,430,Drip Coffee,25000.0,"Also known as filter coffee, it is made by bre...",https://fake.pacmann.io/img/65.jpg
4,5,72,297,Orange Juice,21000.0,"Freshly squeezed orange juice, sweet and tangy...",https://fake.pacmann.io/img/72.jpg
5,6,24,1212,Biryani,65000.0,A flavorful South Asian rice dish made by laye...,https://fake.pacmann.io/img/24.jpg


In [157]:
# produk dengan jumlah tag terbanyak
max_tag = df_tags.product_id.value_counts().max()
# df_tags['product_id']==
tag_count = df_tags.groupby('product_id')['tag'].count().reset_index()

pd.merge(tag_count[tag_count['tag']==max_tag],df_products,on='product_id',how='left')

,product_id,tag,name,price,description,category_id,image_url
0,1,6,Japchae,72500.0,A Korean dish made with stir-fried sweet potat...,1,https://fake.pacmann.io/img/1.jpg
1,2,6,Frozen Tater Tots,35000.0,"Small, grated potatoes formed into rounds and ...",2,https://fake.pacmann.io/img/2.jpg
2,5,6,Mee Goreng,66000.0,A flavorful and spicy stir-fried noodle dish f...,1,https://fake.pacmann.io/img/5.jpg
3,6,6,Frozen Pancakes,58000.0,"Pre-cooked pancakes, ready to heat and serve, ...",2,https://fake.pacmann.io/img/6.jpg
4,8,6,Chicken Shawarma,92000.0,"A Middle Eastern dish made with marinated, spi...",3,https://fake.pacmann.io/img/8.jpg
5,21,6,Sushi Rice,67500.0,"Sticky, vinegared rice used to make sushi roll...",6,https://fake.pacmann.io/img/21.jpg
6,24,6,Biryani,65000.0,A flavorful South Asian rice dish made by laye...,6,https://fake.pacmann.io/img/24.jpg
7,27,6,Chicken Souvlaki,82500.0,A Greek dish made with marinated chunks of chi...,3,https://fake.pacmann.io/img/27.jpg
8,28,6,Matcha,24000.0,"A powdered green tea from Japan, whisked into ...",5,https://fake.pacmann.io/img/28.jpg
9,33,6,Duck Confit,105000.0,Duck legs slow-cooked in their own fat until t...,3,https://fake.pacmann.io/img/33.jpg


In [164]:
# top 5 spenders di ecommerce
cart = pd.merge(df_carts,df_cartitems,how='left',on='cart_id')
cart_price = pd.merge(cart,df_products[['product_id','price']],on='product_id')
cart_price['spending'] = cart_price['price'] * cart_price['order_amt']
id_spender = cart_price.groupby('cust_id')['spending'].sum().sort_values(ascending=False)[:5].reset_index()
pd.merge(id_spender,df_customers,on='cust_id',how='left')

,cust_id,spending,name,phone,address,city,province
0,223,20147000.0,CV Anggriawan,+62 (044) 607 3937,Gg. Moch. Ramdan No. 037,Jakarta Timur,Jakarta
1,780,17940500.0,UD Wibowo (Persero) Tbk,+62 (014) 127-7797,Jalan Cikutra Timur No. 6,Jakarta Timur,Jakarta
2,807,15043500.0,PT Padmasari,(090) 036-6583,Gg. Jamika No. 435,Jakarta Timur,Jakarta
3,54,14219500.0,PD Yolanda Tbk,+62 (529) 435-1293,Gang Pelajar Pejuang No. 7,Jakarta Timur,Jakarta
4,13,14098000.0,Perum Sitorus Tampubolon (Persero) Tbk,+62 (0462) 159 6542,Jalan Antapani Lama No. 2,Jakarta Timur,Jakarta


In [169]:
# di bulan apa yang paling banyak untuk melakukan transaksi
df_carts['trx_date'] = pd.to_datetime(df_carts['trx_date'])
df_carts['month'] = df_carts['trx_date'].dt.month
df_carts.month.value_counts()

10    931
11    885
9     855
8     853
7     846
5     811
6     795
4     773
3     746
1     715
2     675
12    404
Name: month, dtype: int64

In [171]:
# tipe campaign apa yang paling sering dilakukan
df_campaign['campaign_type'].value_counts()

Product     6
Category    5
Name: campaign_type, dtype: int64

,campaign_id,name,campaign_type,start_date,duration
0,1,Festival Makanan Sehat,Category,2023-01-01,31 days
1,2,Bulan Cita Rasa Nusantara,Product,2023-02-01,89 days
2,3,Mei Festival Makanan Lokal,Category,2023-05-01,61 days
3,4,Festival Pedas Juli,Category,2023-07-01,62 days
4,5,Bulan Makanan Manis,Category,2023-09-01,61 days
5,6,November Nikmat,Product,2023-11-01,92 days
6,7,Festival Makanan Laut Februari,Product,2024-02-01,29 days
7,8,Maret Rasa Pedas,Product,2024-03-01,92 days
8,9,Juni Hidangan Segar,Category,2024-06-01,61 days
9,10,Agustus Kuliner Kreatif,Product,2024-08-01,31 days


In [177]:
# campaign apa yang memiliki durasi paling lama sebelum campaign berikutnya dimulai
df_campaign['start_date'] = pd.to_datetime(df_campaign['start_date'])
df_campaign['duration'] = df_campaign.start_date.shift(-1) - df_campaign.start_date
df_campaign.loc[df_campaign.duration.idxmax()]

campaign_id                        6
name                 November Nikmat
campaign_type                Product
start_date       2023-11-01 00:00:00
duration            92 days 00:00:00
Name: 5, dtype: object

In [197]:
# campaign mana yang paling banyak promosikan produk, perhatikan bahwa satu kategori memuat beberapa produk
max_id = df_campaign_items.groupby('campaign_id')['campaign_item'].count().idxmax()
prod_count = df_campaign_items.groupby('campaign_id')['campaign_item'].count().sort_values(ascending=False).reset_index()
# df_campaign[df_campaign['campaign_id']==max_id]
pd.merge(prod_count,df_campaign,on='campaign_id',how='left').iloc[0,:]

campaign_id                                 11
campaign_item                                9
name             September Kuliner Khas Daerah
campaign_type                          Product
start_date                 2024-09-01 00:00:00
duration                                   NaT
Name: 0, dtype: object

In [200]:
# kategori produk dengan total produk berbeda terbanyak
df_products.groupby('category_id')['product_id'].count().sort_values(ascending=False)

category_id
2    41
3    29
5    22
1    20
4    20
6    20
Name: product_id, dtype: int64

In [221]:
# kategori produk favorit untuk tiap bulannya berdasarkan jumlah transaksi
cart = pd.merge(df_carts,df_cartitems,how='left',on='cart_id')
cart = pd.merge(cart,df_products[['product_id','category_id']],how='left',on='product_id')
cart['trx_date'] = pd.to_datetime(cart['trx_date'])
cart['month'] = cart['trx_date'].dt.month
group_month = cart.groupby(['month','category_id'])
group_month = group_month['order_amt'].count().reset_index()

group_month.loc[group_month.groupby(['month'])['order_amt'].idxmax()]

,month,category_id,order_amt
1,1,2,541
7,2,2,499
13,3,2,498
19,4,2,555
25,5,2,548
31,6,2,619
37,7,2,601
43,8,2,616
49,9,2,585
55,10,2,641


In [230]:
# product apa yang paling sering di campaign pada tipe campaign produk dan berapa kali
num = df_campaign_items[df_campaign_items['campaign_type']=='Product']['campaign_item'].value_counts().max()
most_camp = df_campaign_items[df_campaign_items['campaign_type']=='Product']['campaign_item'].value_counts().reset_index()
most_camp[most_camp['campaign_item']==num]

,index,campaign_item
0,Pilaf,2
1,Flat White,2
2,Lasagna,2
3,Jasmine Tea,2


In [297]:
# customer yang telah lama tidak melakukan transaksi kembali (durasinya variabel dalam hari)
from datetime import datetime
# datetime.today()
last_trx = df_carts.loc[df_carts.groupby('cust_id')['trx_date'].idxmax()]
last_trx['duration'] = datetime.today() - last_trx['trx_date']
last_trx['duration'] = last_trx['duration'].dt.days
last_trx[['cust_id','trx_date','duration']].sort_values(by='duration',ascending=False)


,cust_id,trx_date,duration
58,212,2023-01-06,706
87,667,2023-01-10,702
180,262,2023-01-20,692
269,718,2023-01-30,682
323,923,2023-02-05,676
...,...,...,...
9286,544,2024-11-29,13
9277,227,2024-11-29,13
9287,339,2024-11-29,13
9281,737,2024-11-29,13


In [ ]:
# customer yang melakukan transaksi lebih dari satu kali pada hari yang sama
daily_multiple = df_carts[df_carts.duplicated(subset=['cust_id','trx_date'],keep=False)].sort_values(by=['cust_id','trx_date'])
df_customers[df_customers['cust_id'].isin(daily_multiple['cust_id'].unique())]

,cust_id,name,phone,address,city,province
0,1,PT Nuraini Tbk,(070) 602-8828,Gang Tebet Barat Dalam No. 5,Bekasi,Jawa Barat
12,13,Perum Sitorus Tampubolon (Persero) Tbk,+62 (0462) 159 6542,Jalan Antapani Lama No. 2,Jakarta Timur,Jakarta
28,29,PD Prasetya (Persero) Tbk,+62-0264-288-5963,Jl. Cikapayang No. 48,Depok,Jawa Barat
37,38,Perum Megantara Hidayat,+62 (0965) 632 1526,Gang Rawamangun No. 3,Jakarta Timur,Jakarta
41,42,CV Aryani Hutagalung,+62 (072) 078-4275,Jalan Pelajar Pejuang No. 5,Jakarta Barat,Jakarta
...,...,...,...,...,...,...
904,905,UD Hutapea Adriansyah (Persero) Tbk,+62-0792-471-4612,Jalan Abdul Muis No. 4,Jakarta Timur,Jakarta
914,915,UD Prasasta Suryatmi Tbk,+62 (29) 879 0478,Jl. S. Parman No. 950,Bekasi,Jawa Barat
919,920,PD Sitorus Yuniar (Persero) Tbk,+62 (0494) 605-8368,Gang Rajawali Barat No. 9,Bekasi,Jawa Barat
926,927,PD Wibisono Saptono,(0778) 671 0939,Gang Rumah Sakit No. 80,Jakarta Barat,Jakarta


In [287]:
# jumlah customer baru (belum pernah melakukan transaksi sebelumnya ) tiap bulannya
monthly_new = df_carts.loc[df_carts.groupby('cust_id')['trx_date'].idxmin()]
cust_new = monthly_new.month.value_counts().sort_index().reset_index().rename(columns={'month':'new_cust','index':'month'})
cust_new

,month,new_cust
0,1,244
1,2,141
2,3,127
3,4,94
4,5,77
5,6,50
6,7,53
7,8,43
8,9,20
9,10,40


In [288]:
# bandingkan jumlah customer baru dan customer lama (pernah melakukan transaksi sebelumnya) tiap bulannya
old_cust = df_carts.loc[~df_carts.index.isin(monthly_new.index)]
cust_old = old_cust.month.value_counts().sort_index().reset_index().rename(columns={'month':'cust','index':'month'})
cust_old

,month,cust
0,1,471
1,2,534
2,3,619
3,4,679
4,5,734
5,6,745
6,7,793
7,8,810
8,9,835
9,10,891


In [289]:
ratio_cust = pd.merge(cust_new,cust_old,on='month')
ratio_cust['ratio'] = ratio_cust['new_cust']/ratio_cust['cust']
ratio_cust

,month,new_cust,cust,ratio
0,1,244,471,0.518047
1,2,141,534,0.264045
2,3,127,619,0.205170
3,4,94,679,0.138439
4,5,77,734,0.104905
5,6,50,745,0.067114
6,7,53,793,0.066835
7,8,43,810,0.053086
8,9,20,835,0.023952
9,10,40,891,0.044893


# RECOMMENDER SYSTEM

In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from collections import Counter
from datetime import datetime, timezone, timedelta
tzinfo = timezone(timedelta(hours=+7.0))

def init_dest_engine():
    cred = {
        'host': "ep-square-pond-a5h66kcl.us-east-2.aws.neon.tech",
        'user': "neondb_owner",
        'pass': "Kd2w6OvIiUtr",
        'db': "neondb",
        'port': 5432
    }

    uri = f"postgresql://{cred['user']}:{cred['pass']}@{cred['host']}:{cred['port']}/{cred['db']}?sslmode=require"
    conn = create_engine(uri,echo=True)

    return conn

def get_jaccard_similarity(a:list,b:list):
    c = set(a).intersection(set(b))
    return float(len(c))/(len(a)+len(b)-len(c))

def similarity_products(p, n_result:int, dataset:dict):
    result_dict = dict()
    timer_dict = dict()
    for k,v in dataset.items():
        if v == dataset[p]:
            continue
        else:
            result_dict[k] = get_jaccard_similarity(dataset[p],v)
            timer_dict[k] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    counter = Counter(result_dict)

    # formatting
    recommend = list(dict(counter.most_common(n_result)).keys())
    algo    = ['Similar Product']*n_result
    id      = [p]*n_result
    calc_date= [timer_dict[key] for key in recommend]

    return {'product_id':id,
            'recommendation_id':recommend,
            'algorithm':algo,
            'calculation_date':calc_date}



def relatable_products(p,n_result:int,dataset:dict):
    result_list = []
    
    for val in dataset.values():
        if p in val:
            val.remove(p)
            result_list.extend(val)
    
    counter = Counter(result_list)
    
    # formatting
    recommend = list(dict(counter.most_common(n_result)).keys())
    algo    = ['Related Product']*n_result
    id      = [p]*n_result
    calc_date= [datetime.now().strftime("%Y-%m-%d %H:%M:%S")]*n_result

    return {'product_id':id,
            'recommendation_id':recommend,
            'algorithm':algo,
            'calculation_date':calc_date}



def recommender_system(id,n_result:int,engine):
    df_cartitems = pd.read_sql_table(table_name = 'cart_items',con = engine.connect())
    df_tags = pd.read_sql_table(table_name = 'tags',con = engine.connect())
    engine.dispose()

    df_tags['tag_id'] = df_tags['tag_id'].map(lambda x:x.split('-')[-1])
    tag_dict = df_tags.groupby('product_id')['tag_id'].agg(list).to_dict()
    cart_dict = df_cartitems.groupby('cart_id')['product_id'].agg(list).to_dict()

    d1 = similarity_products(id, n_result, dataset=tag_dict)
    d2 = relatable_products(id,n_result, dataset=cart_dict)

    return pd.concat([pd.DataFrame(d1),pd.DataFrame(d2)],ignore_index=True)


def load_process(data: pd.DataFrame, table_name: str, schema_name: str) -> None:

    try:
        conn = init_dest_engine()
        data.to_sql(name=table_name,con=conn,if_exists="append",index=False,schema=schema_name)
        pass

    except Exception as e:
        raise Exception(e)

    finally:
        conn.dispose()


# RUN
conn = init_dest_engine()
df_rec = recommender_system(42,5,conn)
load_process(data = df_rec, table_name = "ProductRecommendations", schema_name = "dev")

2024-12-12 17:49:52,531 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-12-12 17:49:52,536 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-12 17:49:53,016 INFO sqlalchemy.engine.Engine select current_schema()
2024-12-12 17:49:53,019 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-12 17:49:53,492 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-12-12 17:49:53,493 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-12 17:49:54,023 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-12 17:49:54,025 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname